### Imports

In [ ]:
import os
import torch
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
import numpy as np
from tqdm.notebook import tqdm
%matplotlib notebook 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

### Ground truth map mesh

In [ ]:
verts, faces, aux = load_obj(os.path.join('../data/meshes/simple_cave_01.obj'))

# verts is a FloatTensor of shape (V, 3) where V is the number of vertices in the mesh
# faces is an object which contains the following LongTensors: verts_idx, normals_idx and textures_idx
# For this tutorial, normals and textures are ignored.
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)

# We construct a Meshes structure for the target mesh
mesh = Meshes(verts=[verts], faces=[faces_idx])

### Point cloud to compare

In [ ]:
def plot_pointcloud(inputs, title=""):
    assert isinstance(inputs, Meshes) or isinstance(inputs, Pointclouds) or isinstance(inputs, torch.Tensor)
    if isinstance(inputs, Meshes):
        # Sample points uniformly from the surface of the mesh.
        points = sample_points_from_meshes(inputs, 5000)
    elif isinstance(inputs, Pointclouds):
        points = inputs.points_list()[0]
    elif isinstance(inputs, torch.Tensor):
        points = inputs
    x, y, z = points.clone().detach().cpu().squeeze().unbind(1)    
    fig = plt.figure(figsize=(5, 5))
    ax = Axes3D(fig)
    ax.scatter3D(x, z, -y)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    ax.set_title(title)
    ax.view_init(190, 30)
    plt.show()

In [ ]:
torch.manual_seed(0)
points = sample_points_from_meshes(mesh, 5000)
# points = verts[None][:, torch.randint(verts.shape[0], (5000,)), :]
pcl = Pointclouds(points)

In [ ]:
plot_pointcloud(mesh)

In [ ]:
plot_pointcloud(pcl)

### Compare mesh to point cloud

In [ ]:
# Compare mesh and point cloud:
# https://pytorch3d.readthedocs.io/en/latest/modules/loss.html#pytorch3d.loss.point_mesh_edge_distance
from pytorch3d.loss import point_mesh_edge_distance, point_mesh_face_distance

In [ ]:
# point cloud is simply obtained from mesh vertices
with torch.no_grad():
    for s in range(500, 2001, 500):
        points = verts[None][:, torch.randint(verts.shape[0], (s,)), :]
        pcl = Pointclouds(points)
        # plot_pointcloud(pcl)
        print('Num sampled points:', s)
        print('EPPE:', point_mesh_edge_distance(meshes=mesh, pcls=pcl).sqrt().item())
        print('\n')

        print('FPPF:', point_mesh_face_distance(meshes=mesh, pcls=pcl).sqrt().item())
        print('-'*30)

In [ ]:
# point cloud is sampled from mesh surface
with torch.no_grad():
    for s in range(500, 2001, 500):
        points = sample_points_from_meshes(mesh, s)
        pcl = Pointclouds(points)
        # plot_pointcloud(pcl)
        print('Num sampled points:', s)
        print('EPPE:', point_mesh_edge_distance(meshes=mesh, pcls=pcl).sqrt().item())
        print('\n')

        print('FPPF:', point_mesh_face_distance(meshes=mesh, pcls=pcl).sqrt().item())
        print('-'*30)